In [2]:
import pandas as pd
import numpy as np
from scipy import stats

**Задача 1. Оценка вероятностей**

40% заказов имеют стоимость больше 1000 рублей каждый. Допустим, новый заказ стоит больше 1000 рублей с вероятностью 0.4. Оцените вероятность того, что из 100 новых заказов 50 или более стоят дороже 1000 рублей.

Ответ округлите с точность до 3 знака после точки.



***Решение***

Используем распределение Бернулли, т.к идет речь идет о вероятности наступления события. Если считать, что каждый заказ независим от других,можно смоделировать данное событие с помощью биномиального распределения.


In [23]:
#Вариант решения 1

n = 100
p = 0.4
n_new = 50 #порог кол-ва заказов

mu = n * p
sigma = (n * p * (1-p)) ** 0.5

#0.5 - поправка на непрерывность-применяется, когда мы приближаем биномиальное распределение (дискретное) 
# нормальным распределением (непрерывным)
z = (n_new - mu - 0.5)/sigma
#z - стандартная нормальная случ величина
print('Вер-ть P(X) >= 50: ', round(1-stats.norm.cdf(z),3)) 

Вер-ть P(X) >= 50:  0.026


In [24]:
#Вариант решения 2

# Cгенерируем достаточно большое количество исходов
values = np.random.binomial(100, 0.4, size=1000000)
# Оценим вероятность того, что 50 или более заказов стоят дороже 1000
answer = (values >= 50).mean()
print('Вер-ть P(X) >= 50: ', round(answer, 3))

Вер-ть P(X) >= 50:  0.027


**Задача 2. Популярные товары**

 Определите топ-3 товара по суммарной выручке. Для решения используйте данные из файла `2022-04-01/2022-04-01T12_df_sales_detail.csv`.

In [29]:
df = pd.read_csv('./2022-04-01T12_df_sales_detail.csv')

df.groupby('good').agg(revenue=('price', 'sum')).reset_index().sort_values(by='revenue', ascending=False).head(3)

,good,revenue
3,chefs pizza,24558240
6,double pepperoni pizza,22558380
5,chicken bbq pizza,17622540


**Задача 3. Время от захода на сайт до покупки**
  

Оцените средний срок, который проходит от захода пользователя на сайт до совершения покупки. Будем считать, что заход на сайт относится к покупке, если он был совершён не ранее, чем за два часа до совершения покупки. Другими словами, для каждой покупки нужно посчитать время, которое проходит от покупки до первого захода на сайт того же пользователя в течение двух часов перед покупкой. И от полученных значений времени посчитать среднее.

Для решения используйте данные из файлов `2022-04-01T12_df_sales.csv` и `2022-04-01T12_df_web_logs.csv`.

Полученный результат переведите в минуты и округлите до целого значения.

Описание статусов логов:

m - страница с меню;
b - страница с корзиной и потдверждением заказа;
p - страница с оплатой заказа;

In [32]:
df_sales = pd.read_csv('2022-04-01T12_df_sales.csv')
df_web_logs = pd.read_csv('2022-04-01T12_df_web_logs.csv')

display(df_sales.sample(3))
print('*******')
display(df_web_logs.sample(3))

,sale_id,date,count_pizza,count_drink,price,user_id
141137,1141138,2022-03-14 16:29:14,1,1,930,c3052e
47681,1047682,2022-02-17 11:49:12,1,0,720,c08dfb
37058,1037059,2022-02-13 19:27:01,1,0,600,ffff8f


*******


,user_id,page,date,load_time
698480,8f72ed,m,2022-02-23 18:58:11,74.4
1375445,560c37,m,2022-03-15 08:23:22,62.2
428773,78fc29,m,2022-02-16 11:48:40,76.1


In [43]:
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_sales['dt'] = df_sales['date'].dt.date

df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])
df_web_logs['dt'] = pd.to_datetime(df_web_logs['date']).dt.date

In [49]:
df_uni = df_sales.merge(df_web_logs, on=['dt', 'user_id'], suffixes=['_sales', '_logs'])
df_uni.shape

(1628116, 10)

In [51]:
df_uni['diff_datetime'] = (df_uni['date_sales']-df_uni['date_logs']).dt.total_seconds() / 3600
#оставляем только строки, где разница между покупкой и заходом на сайт меньше 2 часов
df_uni = df_uni[df_uni['diff_datetime']<=2]
#находим минимальную дату входа для каждого заказа
min_datetime_enter_df = df_uni.groupby(['sale_id', 'date_sales']).agg(min_datetime_enterance = ('date_logs', 'min')).reset_index()

In [57]:
print('Сколько в среднем проходит времени от захода на сайт до покупки: ',
       round(((min_datetime_enter_df['date_sales']-min_datetime_enter_df['min_datetime_enterance']).dt.total_seconds() / 60).mean()))

Сколько в среднем проходит времени от захода на сайт до покупки:  17


**Задача 4. Знакомство с задачами на код**
  

В заданиях на код вам нужно отправить в систему проверки код с решением. Шаблон для решения будет предоставлен в описании задания. Проверка решения запускается в докер-контейнере, в котором доступны следующие версии библиотек:

`numpy=1.19.5`
`pandas==1.2.3`
`scipy==1.5.4`

После окончания проверки отобразятся количество набранных баллов и список пройденных проверок. Если при проверке возникла ошибка, то будет выведено её описание.

Напишите функцию sum_two_values. Описание функции есть в шаблоне решения ниже. Скопируйте код шаблона в окно ввода решения, допишите недостающие части, нажмите кнопку "Отправить решение".

Шаблон решения

`def sum_two_values(a, b):`
    `"""Возвращает сумму чисел."""`
    `# YOUR_CODE_HERE`

In [ ]:
def sum_two_values(a, b):
    """Возвращает сумму чисел."""
    c = a + b
    return c

**Задача 5. Функция для фильтрации данных**
  

Для оценки результатов экспериментов нам нужно выбрать данные, полученные во время эксперимента.

Напишите функцию `get_data_subset`.



In [6]:
from datetime import datetime

import pandas as pd


def get_data_subset(df, begin_date, end_date, user_ids=None, columns=None):
    """Возвращает подмножество данных.

    :param df (pd.DataFrame): таблица с данными, обязательные столбцы: 'date', 'user_id'.
    :param begin_date (datetime.datetime | None): дата начала интервала с данными.
        Пример, df[df['date'] >= begin_date].
        Если None, то фильтровать не нужно.
    :param end_date (datetime.datetime | None): дата окончания интервала с данными.
        Пример, df[df['date'] < end_date].
        Если None, то фильтровать не нужно.
    :param user_ids (list[str] | None): список user_id, по которым нужно предоставить данные.
        Пример, df[df['user_id'].isin(user_ids)].
        Если None, то фильтровать по user_id не нужно.
    :param columns (list[str] | None): список названий столбцов, по которым нужно предоставить данные.
        Пример, df[columns].
        Если None, то фильтровать по columns не нужно.

    :return df (pd.DataFrame): датафрейм с подмножеством данных.
    """
    # YOUR_CODE_HERE
    new_df = df.copy(deep=True)
    
    if begin_date is not None:
        new_df = new_df[new_df['date']>=begin_date]

    if end_date is not None:
        new_df = new_df[new_df['date']<end_date]

    if user_ids is not None:
        new_df = new_df[new_df['user_id'].isin(user_ids)]

    if columns is not None:
        new_df = new_df[columns]

    return new_df
    

In [7]:
df = pd.DataFrame({
    'date': [datetime(2022, 1, 5), datetime(2022, 1, 7)],
    'user_id': ['1', '2'],
})
new_df = get_data_subset(df, datetime(2022, 1, 1), datetime(2022, 1, 6))

In [8]:
new_df

,date,user_id
0,2022-01-05,1


In [5]:
df

,date,user_id
0,2022-01-05,1
1,2022-01-07,2


**Задача 6. Функции вычисления метрик**

Напишите функции get_response_time, get_revenue_web и get_revenue_all для вычисления метрик «revenue (web)», «revenue (all)» и «response time».

In [39]:
import pandas as pd

from datetime import datetime

def get_data_subset(df, begin_date, end_date, user_ids=None, columns=None):
    if begin_date:
        df = df[df['date'] >= begin_date]
    if end_date:
        df = df[df['date'] < end_date]
    if user_ids:
        df = df[df['user_id'].isin(user_ids)]
    if columns:
        df = df[columns]
    return df.copy()


def get_response_time(df_web_logs, begin_date, end_date):
    """Вычисляет значения времени обработки запроса сервером.

    Нужно вернуть значения user_id и load_time из таблицы df_web_logs,
    отфильтрованные по дате.
    Считаем, что запросы обрабатываются независимо, поэтому группировать
    по user_id не нужно.

    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
    столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для
    фильтрации данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """

    new_df  = get_data_subset(df_web_logs, begin_date, end_date, None, ['user_id', 'load_time']).copy(deep=True)
    new_df = new_df.rename(columns={'load_time':'metric'})

    return new_df[['user_id', 'metric']]


def get_revenue_web(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт в указанный период.

    Эти данные нужны для экспериментов на сайте, когда в эксперимент
    попадают только те, кто заходил на сайт во время эксперимента.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный
    период для пользователей, заходивших на сайт в указанный период.
    Если пользователь зашёл на сайт и ничего не купил, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    df_sales['date'] = pd.to_datetime(df_sales['date'])
    df_sales['dt'] = df_sales['date'].dt.date

    df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])
    df_web_logs['dt'] = pd.to_datetime(df_web_logs['date']).dt.date

    df_web_logs = get_data_subset(df_web_logs, begin_date, end_date, None, ['dt', 'user_id']).drop_duplicates()
    df_sales = get_data_subset(df_sales, begin_date, end_date, None, ['dt', 'user_id', 'price'])

    df_uni = df_web_logs.merge(df_sales, on=['dt', 'user_id'], how='left')
    df_uni['price'] = df_uni['price'].fillna(0)

    df_metric = df_uni.groupby('user_id').agg(metric=('price', 'sum')).reset_index()

    return df_metric

    
def get_revenue_all(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт до end_date.

    Эти данные нужны, например, для экспериментов с рассылкой по email,
    когда в эксперимент попадают те, кто когда-либо оставил нам свои данные.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный период
    для пользователей, заходивших на сайт до end_date.
    Если пользователь ничего не купил за указанный период, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    df_sales['date'] = pd.to_datetime(df_sales['date'])
    df_sales['dt'] = df_sales['date'].dt.date

    df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])
    df_web_logs['dt'] = pd.to_datetime(df_web_logs['date']).dt.date

    df_web_logs = get_data_subset(df_web_logs, None, end_date, None, ['dt', 'user_id']).drop_duplicates()
    df_sales = get_data_subset(df_sales, begin_date, end_date, None, ['dt', 'user_id', 'price'])

    df_uni = df_web_logs.merge(df_sales, on=['dt', 'user_id'], how='left')
    df_uni['price'] = df_uni['price'].fillna(0)

    df_metric = df_uni.groupby('user_id').agg(metric=('price', 'sum')).reset_index()

    return df_metric

In [40]:
df_sales = pd.DataFrame({
    'date': [datetime(2022, 3, day, 11) for day in range(11, 14)],
    'price': [1100, 900, 1500],
    'user_id': ['1', '2', '1'],
})
df_web_logs = pd.DataFrame({
    'date': [datetime(2022, 3, day, 11) for day in range(10, 14)],
    'load_time': [80.8, 90.1, 15.8, 19.7],
    'user_id': ['3', '1', '2', '1'],
})
begin_date = datetime(2022, 3, 11, 9)
end_date = datetime(2022, 4, 11, 9)

In [41]:
####Проверка

In [42]:
response_time = get_response_time(df_web_logs, begin_date, end_date)
# response_time = pd.DataFrame({'user_id': ['1', '2', '1'], 'metric': [90.1, 15.8, 19.7],})
response_time

,user_id,metric
1,1,90.1
2,2,15.8
3,1,19.7


In [43]:
revenue_web = get_revenue_web(df_sales, df_web_logs, begin_date, end_date)
# revenue_web = pd.DataFrame({'user_id': ['1', '2'], 'metric': [2600, 900],})

revenue_web

,user_id,metric
0,1,2600
1,2,900


In [44]:
revenue_all = get_revenue_all(df_sales, df_web_logs, begin_date, end_date)
# revenue_all = pd.DataFrame({'user_id': ['1', '2', '3'], 'metric': [2600, 900, 0],})
revenue_all

,user_id,metric
0,1,2600.0
1,2,900.0
2,3,0.0
